<a href="https://colab.research.google.com/github/malcolmlett/ml-learning/blob/main/Gradient_understanding_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# For a blog post on gradients within deep neural networks

In [2]:
import numpy as np
import tensorflow as tf
import math
import matplotlib.pyplot as plt

## Working it out
First a set of experiments to prove the basics.


In [ ]:
# extremely simple model - with overly simple loss function
# conclusions:
#  - dY/dW is proportional to n
#  - dJ/dW is averaged across all n if that's was the loss function does, however
#  - dJ/dW will be summed across n, or any other operation, depending how the loss is computed.
W = tf.Variable(tf.ones(shape=(3,3)), dtype=tf.float32)
X = tf.ones(shape=(15,3))

with tf.GradientTape(persistent=True) as tape:
  Y_pred = tf.matmul(X, W)
  loss1 = tf.reduce_mean(Y_pred)
  loss2 = tf.reduce_sum(Y_pred)

print(f"dY/dW: {tape.gradient(Y_pred, W)}")
print(f"dJ1/dW: {tape.gradient(loss1, W)}")
print(f"dJ2/dW: {tape.gradient(loss2, W)}")

dY/dW: [[15. 15. 15.]
 [15. 15. 15.]
 [15. 15. 15.]]
dJ1/dW: [[0.33333334 0.33333334 0.33333334]
 [0.33333334 0.33333334 0.33333334]
 [0.33333334 0.33333334 0.33333334]]
dJ2/dW: [[15. 15. 15.]
 [15. 15. 15.]
 [15. 15. 15.]]


In [ ]:
# extremely simple model - with more realistic loss function
# conclusions:
#  - dY/dW is proportional to n
#  - dJ/dW is averaged across all n if that's was the loss function does, however
#  - dJ/dW will be summed across n, or any other operation, depending how the loss is computed.
W = tf.Variable(tf.ones(shape=(3,3)), dtype=tf.float32)
X = tf.ones(shape=(15,3))
Y = tf.ones(shape=(15,3))

with tf.GradientTape(persistent=True) as tape:
  Y_pred = tf.matmul(X, W)
  loss1 = tf.reduce_mean(0.5*(Y_pred - Y)**2)    # reflects the common MSE loss
  loss2 = tf.reduce_sum(0.5*(Y_pred - Y)**2)     # for comparison if we remove the 1/n from the front

# J = 1/n * sum(1/2 * (Y_pred - Y)**2)         [n = batch size, d = depth]
# dJ/dW = dJ/dY . dY/dW
# dJ/dY = 1/n * sum(Y_pred - Y)

print(f"X:{X[:5,:]}")
print(f"Y_pred:{Y_pred[:5,:]}")
print(f"error:{(Y_pred - Y)[:5,:]}")
print(f"loss1:{loss1}")
print(f"loss2:{loss2}")
print("---")
print(f"dJ/dW = dJ/dY . dY/dW")
print(f"dJ1/dY: {tape.gradient(loss1, Y_pred)[:5,:]}")  # just first 5 rows out of 15 (all same), note: 0.04444445 = 1/15*1/3*2
print(f"dJ2/dY: {tape.gradient(loss2, Y_pred)[:5,:]}")  # just first 5 rows out of 15 (all same)
print(f"dY/dW: {tape.gradient(Y_pred, W)}")
print(f"dJ1/dW: {tape.gradient(loss1, W)}")
print(f"dJ2/dW: {tape.gradient(loss2, W)}")

X:[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
Y_pred:[[3. 3. 3.]
 [3. 3. 3.]
 [3. 3. 3.]
 [3. 3. 3.]
 [3. 3. 3.]]
error:[[2. 2. 2.]
 [2. 2. 2.]
 [2. 2. 2.]
 [2. 2. 2.]
 [2. 2. 2.]]
loss1:2.0
loss2:90.0
---
dJ/dW = dJ/dY . dY/dW
dJ1/dY: [[0.04444445 0.04444445 0.04444445]
 [0.04444445 0.04444445 0.04444445]
 [0.04444445 0.04444445 0.04444445]
 [0.04444445 0.04444445 0.04444445]
 [0.04444445 0.04444445 0.04444445]]
dJ2/dY: [[2. 2. 2.]
 [2. 2. 2.]
 [2. 2. 2.]
 [2. 2. 2.]
 [2. 2. 2.]]
dY/dW: [[15. 15. 15.]
 [15. 15. 15.]
 [15. 15. 15.]]
dJ1/dW: [[0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]]
dJ2/dW: [[30. 30. 30.]
 [30. 30. 30.]
 [30. 30. 30.]]


In [ ]:
# How do the out-of-box loss functions work with multi-dimensional outputs?
# conclusions:
#  - out of box MSE loss function takes the mean across the feature dimensions always
#  - out of box MSE loss function usually takes the mean across the sample dimenion too (reduction = default = 'sum_over_batch_size')
#     but it can alternatively take the sum, or just emit all individual values
#
# In other words, for a normal application using MSE, the loss function is computed as follows:
#   J = 1/n * 1/d * sum(over batch: 1/2 * sum(over depth: (Y_pred - Y)**2))         [n = batch size, d = depth]
#   dJ/dW = dJ/dY . dY/dW
#   dJ/dY = 1/n * 1/d * sum_b(sum_d(Y_pred - Y)) = mean(Y_pred - Y, axis=all)

W = tf.Variable(tf.ones(shape=(3,3)), dtype=tf.float32)
X = tf.ones(shape=(15,3))
Y = tf.ones(shape=(15,3))
Y_pred = tf.zeros(shape=(15,3))
print(f"Loss from Y_pred shape {Y_pred.shape}: {tf.keras.losses.MeanSquaredError()(Y, Y_pred)}  (standard MSE, reduction = default = 'sum_over_batch_size')")
print(f"Loss from Y_pred shape {Y_pred.shape}: {tf.keras.losses.MeanSquaredError(reduction='sum')(Y, Y_pred)} (reduction = 'sum')")
print(f"Loss from Y_pred shape {Y_pred.shape}: {tf.keras.losses.MeanSquaredError(reduction=None)(Y, Y_pred)} (reduction = None)")

print(f"---")

Y_pred = np.zeros(shape=(15,3))
Y_pred[:,2] = 1
print(f"Loss from Y_pred shape {Y_pred.shape}: {tf.keras.losses.MeanSquaredError()(Y, Y_pred)}  (standard MSE, reduction = default = 'sum_over_batch_size')")
print(f"Loss from Y_pred shape {Y_pred.shape}: {tf.keras.losses.MeanSquaredError(reduction='sum')(Y, Y_pred)} (reduction = 'sum')")
print(f"Loss from Y_pred shape {Y_pred.shape}: {tf.keras.losses.MeanSquaredError(reduction=None)(Y, Y_pred)} (reduction = None)")


Loss from Y_pred shape (15, 3): 1.0  (standard MSE, reduction = default = 'sum_over_batch_size')
Loss from Y_pred shape (15, 3): 15.0 (reduction = 'sum')
Loss from Y_pred shape (15, 3): [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.] (reduction = None)
---
Loss from Y_pred shape (15, 3): 0.6666666865348816  (standard MSE, reduction = default = 'sum_over_batch_size')
Loss from Y_pred shape (15, 3): 10.0 (reduction = 'sum')
Loss from Y_pred shape (15, 3): [0.6666667 0.6666667 0.6666667 0.6666667 0.6666667 0.6666667 0.6666667
 0.6666667 0.6666667 0.6666667 0.6666667 0.6666667 0.6666667 0.6666667
 0.6666667] (reduction = None)


In [ ]:
# Same simple model as above now using out-of-box loss function
# conclusions:
#  We got almost exactly the same thing as when doing it manually. The only difference is the '1/2' factor. Yay.
W = tf.Variable(tf.ones(shape=(3,3)), dtype=tf.float32)
X = tf.ones(shape=(15,3))
Y = tf.ones(shape=(15,3))
loss_fn = tf.keras.losses.MeanSquaredError()  # works the same as my tf.reduce_mean(0.5*(Y_pred - Y)**2) from above, BUT without the '1/2' factor

with tf.GradientTape(persistent=True) as tape:
  Y_pred = tf.matmul(X, W)
  loss = loss_fn(Y, Y_pred)

print(f"X:{X[:5,:]}")
print(f"Y_pred:{Y_pred[:5,:]}")
print(f"error:{(Y_pred - Y)[:5,:]}")
print(f"loss:{loss}")
print("---")
print(f"dJ/dW = dJ/dY . dY/dW")
print(f"dJ/dY: {tape.gradient(loss, Y_pred)[:5,:]}")  # just first 5 rows out of 15 (all same), note: 0.0888889 = 1/15*1/3*2*2
print(f"dY/dW: {tape.gradient(Y_pred, W)}")
print(f"dJ/dW: {tape.gradient(loss, W)}")
print("---")
print("jacobian:")
print(f"dJ/dY: {tape.jacobian(loss, Y_pred).shape} - {tape.jacobian(loss, Y_pred)[:5,:]}")  # just first 5 rows out of 15 (all same), note: 0.0888889 = 1/15*1/3*2*2

X:[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
Y_pred:[[3. 3. 3.]
 [3. 3. 3.]
 [3. 3. 3.]
 [3. 3. 3.]
 [3. 3. 3.]]
error:[[2. 2. 2.]
 [2. 2. 2.]
 [2. 2. 2.]
 [2. 2. 2.]
 [2. 2. 2.]]
loss:4.0
---
dJ/dW = dJ/dY . dY/dW
dJ/dY: [[0.08888889 0.08888889 0.08888889]
 [0.08888889 0.08888889 0.08888889]
 [0.08888889 0.08888889 0.08888889]
 [0.08888889 0.08888889 0.08888889]
 [0.08888889 0.08888889 0.08888889]]
dY/dW: [[15. 15. 15.]
 [15. 15. 15.]
 [15. 15. 15.]]
dJ/dW: [[1.3333334 1.3333334 1.3333334]
 [1.3333334 1.3333334 1.3333334]
 [1.3333334 1.3333334 1.3333334]]
---
jacobian:
dJ/dY: (15, 3) - [[0.0888889 0.0888889 0.0888889]
 [0.0888889 0.0888889 0.0888889]
 [0.0888889 0.0888889 0.0888889]
 [0.0888889 0.0888889 0.0888889]
 [0.0888889 0.0888889 0.0888889]]


In [ ]:
# Same as above but with random values
# Expect to see that dY/dW has each column the same value
W = tf.Variable(tf.random.normal(shape=(3,3)), dtype=tf.float32)
X = tf.random.normal(shape=(15,3))
Y = tf.random.normal(shape=(15,3))
loss_fn = tf.keras.losses.MeanSquaredError()  # works the same as my tf.reduce_mean(0.5*(Y_pred - Y)**2) from above, BUT without the '1/2' factor

with tf.GradientTape(persistent=True) as tape:
  Y_pred = tf.matmul(X, W)
  loss = loss_fn(Y, Y_pred)

print(f"X:{X[:5,:]}")
print(f"Y_pred:{Y_pred[:5,:]}")
print(f"error:{(Y_pred - Y)[:5,:]}")
print(f"loss:{loss}")
print("---")
print(f"dJ/dW = dJ/dY . dY/dW")
print(f"dJ/dY: {tape.gradient(loss, Y_pred)[:5,:]}")  # just first 5 rows out of 15 (all same), note: 0.0888889 = 1/15*1/3*2*2
print(f"dY/dW: {tape.gradient(Y_pred, W)}")
print(f"dJ/dW: {tape.gradient(loss, W)}")

X:[[-0.43365982  0.5280366   1.6713797 ]
 [ 0.7201729   0.8335855   0.8018656 ]
 [-0.47507477 -0.8587969   0.9780858 ]
 [ 1.2738467  -0.4910632   1.3990924 ]
 [-0.23144156  0.04747184 -1.6782101 ]]
Y_pred:[[ 0.5941853  -1.8028502   0.59811175]
 [ 0.09435182 -0.742683    1.1430625 ]
 [-0.05283837 -0.6590447  -2.2933812 ]
 [-0.6865124  -0.48568112 -2.6250665 ]
 [-0.02870353  1.3116801   1.1409246 ]]
error:[[ 1.6950428  -2.5516498   1.9101233 ]
 [ 1.0878788  -0.09027165  0.29277384]
 [ 0.57212424 -0.21477687 -1.2889645 ]
 [ 0.08081627 -0.30215716 -1.3815272 ]
 [-0.39601374  1.8736382   1.751705  ]]
loss:1.9773621559143066
---
dJ/dW = dJ/dY . dY/dW
dJ/dY: [[ 0.07533524 -0.11340666  0.08489437]
 [ 0.04835017 -0.00401207  0.01301217]
 [ 0.02542775 -0.00954564 -0.05728731]
 [ 0.00359183 -0.01342921 -0.06140121]
 [-0.01760061  0.08327281  0.07785356]]
dY/dW: [[ 5.52846     5.52846     5.52846   ]
 [ 0.69284487  0.69284487  0.69284487]
 [-2.5651832  -2.5651832  -2.5651832 ]]
dJ/dW: [[ 0.0111897

## Step by step
Ok, now we seem to consistently be getting successful results. Let's work this through in a logical order.

First, some notation:
* Let $n$ be the number of samples, represented as rows
* Let $l \in 0..L$ refer to a target layer, with layer $0$ representing the input and layer $L$ representing the final output
* Let $Z_l$ be the output matrix from layer $l$, with special cases $Z_0 = X$ and $Z_L = \hat{Y}$
* Let $f_l$ be the number of feature columns at layer $l$, with special cases $f_0$ = columns in $X$ and $f_L$ = columns in $Y$
* Let $W_l \in \mathbb{R}^{f_{l-1}, f_l}$ be the weighs at layer $l$
* Let $J \in \mathbb{R}^{f_L}$ be the loss function

### Single layer network with vector output
The weight gradients at the single layer $l=1$ is given by:
$$\frac{\partial J}{\partial W_1} = \frac{\partial \hat{Y}}{\partial W_1} \cdot \frac{\partial J}{\partial \hat{Y}} = Z_0^T \cdot (\frac{2}{Nf_L}(\hat{Y}-Y))$$
with dimensions:
$$(f_0 \times f_1) = (f_0 \times n) \cdot (n \times f_1)$$

In [22]:
np.random.seed(1) # for consistency
tf.random.set_seed(1) # for consistency
n = 15
fL = 4
W = tf.Variable(tf.random.normal(shape=(3,fL)), dtype=tf.float32)
Z0 = tf.random.normal(shape=(n,3))
Y = tf.random.normal(shape=(n,fL))
loss_fn = tf.keras.losses.MeanSquaredError()  # equivalent to tf.reduce_mean((Y_pred - Y)**2, axis=None) = 1/(N * f_L) * sum((Y_pred-Y)**2)

with tf.GradientTape(persistent=True) as tape:
  Y_pred = tf.matmul(Z0, W)
  loss = loss_fn(Y, Y_pred)

print(f"Z0:{Z0[:5,:]}")
print(f"Y_pred:{Y_pred[:5,:]}...")
print(f"error:{(Y_pred - Y)[:5,:]}...")
print(f"loss:{loss}")
print("---expect---")
dYdW1 = Z0.numpy().T
dJdY = 2/(n*fL)*(Y_pred-Y)
print(f"dY/dW1 = Z0.T: {dYdW1[:,:5]}...<more cols>")
print(f"dJ/dY = 2(n*fL)*error: {dJdY[:5,:]}...")
print(f"dJ/dW1: {np.matmul(dYdW1, dJdY)}")
print("---actual---")
print(f"dY/dW1: {tape.gradient(Y_pred, W)}  <--- DIFFERENT")
print(f"dJ/dY: {tape.gradient(loss, Y_pred)[:5,:]}...")
print(f"dJ/dW1: {tape.gradient(loss, W)}")

Z0:[[ 0.40308788 -1.0880208  -0.06309535]
 [ 1.3365567   0.7117601  -0.48928645]
 [-0.7642213  -1.0372486  -1.2519338 ]
 [ 0.02122428 -0.5513758  -1.7431698 ]
 [-0.33536094 -1.0426675   1.0091382 ]]
Y_pred:[[ 0.92379737  1.7360169   0.09860273 -0.08831309]
 [-2.070418    1.7202142   0.88154966 -0.9705327 ]
 [ 2.81064     0.7384437   0.39943358  1.77825   ]
 [ 1.6248431   1.8298045   1.0509385   1.3398144 ]
 [ 1.0829754  -0.22253022 -0.85192645 -0.2146242 ]]...
error:[[ 1.3808095   2.1428843  -0.629975    0.8046647 ]
 [-2.3830295   0.72592175  2.665812   -0.44852757]
 [ 1.8298397   1.4144287  -0.74618113  1.5721774 ]
 [ 1.8220595   1.2917111   0.2866087   2.1760063 ]
 [ 0.7468337  -1.7827321  -1.5747099  -1.2901129 ]]...
loss:2.4468226432800293
---expect---
dY/dW1 = Z0.T: [[ 0.40308788  1.3365567  -0.7642213   0.02122428 -0.33536094]
 [-1.0880208   0.7117601  -1.0372486  -0.5513758  -1.0426675 ]
 [-0.06309535 -0.48928645 -1.2519338  -1.7431698   1.0091382 ]]...<more cols>
dJ/dY = 2(n*fL

There's a few things to take away.

Firstly, our calculation of the final $\frac{\partial J}{\partial W_1}$ is exactly correct. That's great news.

Secondly, we've reconfirmed how the MSE error produces $\frac{\partial J}{\partial \hat{Y}}$, by way that we get the same result now.

Lastly, the gradient tape provides a very different answer to $\frac{\partial \hat{Y}}{\partial W_1}$. Specifically, it forces the result into the shape of $W_1$. This makes sense if you were wanting to update your $W_1$ based on this gradient, but not if it's intended to be used as part of the chain-rule. In short, the gradient-tape is designed for a very specific purpose, of producing "gradients" for the purpose of "gradient descent/ascent updates". This means that we cannot depend on it for helping us compute all the partial derivates.

A couple more notes:
* You can't use the $\frac{\partial \hat{Y}}{\partial W_1}$ as calculated by the gradient tape in a chain rule calculation. The matrix sizes don't match.
* The gradient tape's `jacobian()` function produces a 3D tensor for the same data, which I also don't know how to use.

### Two-layer network with vector output
Here we have:
* $\hat{Y} = Z_2$

And so, the weight gradients at the first layer $l=1$ is given by:
$$\frac{\partial J}{\partial W_1} = \frac{\partial Z_1}{\partial W_1} \cdot \frac{\partial J}{\partial \hat{Y}} \cdot \frac{\partial \hat{Y}}{\partial Z_1} = \frac{\partial Z_1}{\partial W_1} \cdot \frac{\partial J}{\partial Z_2} \cdot \frac{\partial Z_2}{\partial Z_1} = Z_0^T \cdot (\frac{2}{Nf_L}(\hat{Y}-Y)) \cdot W_2^T$$
with dimensions:
$$(f_0 \times f_1) = (f_0 \times n) \cdot (n \times f_2) \cdot (f_2 \times f_1)$$

And the weight gradients at the second layer $l=2$ is given by:
$$\frac{\partial J}{\partial W_2} = \frac{\partial \hat{Y}}{\partial W_2} \cdot \frac{\partial J}{\partial \hat{Y}} = \frac{\partial Z_2}{\partial W_2} \cdot \frac{\partial J}{\partial Z_2}= Z_1^T \cdot (\frac{2}{Nf_L}(\hat{Y}-Y))$$
with dimensions:
$$(f_1 \times f_2) = (f_1 \times n) \cdot (n \times f_1)$$

In [27]:
np.random.seed(1) # for consistency
tf.random.set_seed(1) # for consistency
n = 15
fL = 4
Z0 = tf.random.normal(shape=(n,2))
W1 = tf.Variable(tf.random.normal(shape=(2,3)), dtype=tf.float32)
W2 = tf.Variable(tf.random.normal(shape=(3,fL)), dtype=tf.float32)
Y = tf.random.normal(shape=(n,fL))
loss_fn = tf.keras.losses.MeanSquaredError()  # equivalent to tf.reduce_mean((Y_pred - Y)**2, axis=None) = 1/(N * f_L) * sum((Y_pred-Y)**2)

with tf.GradientTape(persistent=True) as tape:
  Z1 = tf.matmul(Z0, W1)
  Z2 = tf.matmul(Z1, W2)
  Y_pred = Z2
  loss = loss_fn(Y, Y_pred)

print(f"Z0:{Z0[:5,:]}...")
print(f"Z1:{Z1[:5,:]}...")
print(f"Y_pred:{Y_pred[:5,:]}...")
print(f"error:{(Y_pred - Y)[:5,:]}...")
print(f"loss:{loss}")
print("---expect---")
dZ1dW1 = Z0.numpy().T
dZ2dW2 = Z1.numpy().T
dZ2dZ1 = W2.numpy().T
dJdZ2 = 2/(n*fL)*(Y_pred-Y)
print(f"dZ1/dW1 = Z0.T: {dZ1dW1[:,:5]}...<more cols>")
print(f"dZ2/dW2 = Z1.T: {dZ2dW2[:,:5]}...<more cols>")
print(f"dZ2/dZ1 = W1.T: {dZ2dZ1}")
print(f"dJ/dZ2 = 2(n*fL)*error: {dJdZ2[:5,:]}...")
print(f"dJ/dW1 = dZ1dW1.dJ/dZ2.dZ2/dZ1: {np.matmul(dZ1dW1, np.matmul(dJdZ2, dZ2dZ1))}")
print(f"dJ/dW2 = dZ2dW2.dJ/dZ2: {np.matmul(dZ2dW2, dJdZ2)}")
print("---actual---")
print(f"dZ1/dW1: {tape.gradient(Z1, W1)} <-- DIFFERENT as expected")
print(f"dZ2/dW2: {tape.gradient(Z2, W2)} <-- DIFFERENT as expected")
print(f"dJ/dZ2: {tape.gradient(loss, Y_pred)[:5,:]}...")
print(f"dJ/dW1: {tape.gradient(loss, W1)}")
print(f"dJ/dW2: {tape.gradient(loss, W2)}")

Z0:[[-1.1012203   1.5457517 ]
 [ 0.383644   -0.87965786]
 [-1.2246722  -0.9811211 ]
 [ 0.08780783 -0.20326038]
 [-0.5581562  -0.72054404]]...
Z1:[[ 1.6220962   2.2983549  -0.6868335 ]
 [-1.0210704  -1.0435181   0.4061985 ]
 [-1.8049746   0.634146    0.5573204 ]
 [-0.23627473 -0.24020937  0.09391228]
 [-1.1880339   0.09443104  0.38776952]]...
Y_pred:[[-0.69647235  2.0895483  -3.7058916  -2.7897866 ]
 [ 0.5388256  -0.89670616  1.583328    1.5402212 ]
 [ 1.5697569   0.9881714  -1.8080726   1.3956233 ]
 [ 0.12499745 -0.20618922  0.36403933  0.3557314 ]
 [ 0.9527908   0.31513777 -0.5898303   1.0915031 ]]...
error:[[-2.390489    1.9698552  -2.5474315  -2.9623907 ]
 [ 1.2533219  -1.5863068   2.6741438   2.7267315 ]
 [ 2.871945   -0.23583072 -1.8310329   2.5857396 ]
 [ 2.5590734   0.83327395 -0.561208    0.10472724]
 [-0.23693484 -0.1649693  -0.70998156  1.8163954 ]]...
loss:3.9742300510406494
---expect---
dZ1/dW1 = Z0.T: [[-1.1012203   0.383644   -1.2246722   0.08780783 -0.5581562 ]
 [ 1.5457

Great! That worked as expected.

### Single layer network with activation function as diagonal matrix
Now we'll introduce activation functions by representing them diagonal matrices $S$ s.t. $S_{ii} = \sigma_i$ and $\sigma_i$ is the elementwise scalar multiplicative factor equivalent of the activation function for the given $Z$ input.

Here:
* $A_0 = Z_0 = X$
* $A_L = \hat{Y}$
* $Z_l = X \cdot W_l$
* $A_l = Z_L \cdot S_l$

The weight gradients at the single layer $l=1$ is given by:
$$\frac{\partial J}{\partial W_1} = \frac{\partial Z_1}{\partial W_1} \cdot \frac{\partial J}{\partial (A_1 = \hat{Y})} \cdot \frac{\partial (A_1 = \hat{Y})}{\partial Z_1} = Z_0^T \cdot (\frac{2}{Nf_L}(\hat{Y}-Y)) \cdot S_l^T$$
with dimensions:
$$(f_0 \times f_1) = (f_0 \times n) \cdot (n \times f_1) \cdot (f_1 \times f_1)$$

In [39]:
np.random.seed(1) # for consistency
tf.random.set_seed(1) # for consistency
n = 15
fL = 4
Z0 = tf.random.normal(shape=(n,3))
W1 = tf.Variable(tf.random.normal(shape=(3,fL)), dtype=tf.float32)
S1 = tf.cast(tf.linalg.diag(np.random.normal(size=fL)), dtype=tf.float32)
Y = tf.random.normal(shape=(n,fL))
loss_fn = tf.keras.losses.MeanSquaredError()  # equivalent to tf.reduce_mean((Y_pred - Y)**2, axis=None) = 1/(N * f_L) * sum((Y_pred-Y)**2)

with tf.GradientTape(persistent=True) as tape:
  Z1 = tf.matmul(Z0, W1)
  A1 = tf.matmul(Z1, S1)
  Y_pred = A1
  loss = loss_fn(Y, Y_pred)

print(f"W1:{W1[:5,:]}...")
print(f"S1:{S1[:5,:]}...")
print(f"Z0:{Z0[:5,:]}...")
print(f"Z1:{Z1[:5,:]}...")
print(f"Y_pred:{Y_pred[:5,:]}...")
print(f"error:{(Y_pred - Y)[:5,:]}...")
print(f"loss:{loss}")
print("---expect---")
dZ1dW1 = Z0.numpy().T
dJdA1 = 2/(n*fL)*(Y_pred-Y)
dA1dZ1 = S1.numpy().T
print(f"dZ1/dW1 = Z0.T: {dZ1dW1[:,:5]}...<more cols>")
print(f"dJ/dA1 = 2(n*fL)*error: {dJdA1[:5,:]}...")
print(f"dA1/dZ1 = W1.T: {dA1dZ1}")
print(f"dJ/dW1 = dZ1dW1.dJ/dZ2.dZ2/dZ1: {np.matmul(dZ1dW1, np.matmul(dJdA1, dA1dZ1))}")
print("---actual---")
print(f"dZ1/dW1: {tape.gradient(Z1, W1)} <-- DIFFERENT as expected")
print(f"dJ/dA1: {tape.gradient(loss, Y_pred)[:5,:]}...")
print(f"dA1/dZ1: {tape.gradient(A1, Z1)} <-- DIFFERENT as expected")
print(f"dJ/dW1: {tape.gradient(loss, W1)}")

W1:[[ 0.40308788 -1.0880208  -0.06309535  1.3365567 ]
 [ 0.7117601  -0.48928645 -0.7642213  -1.0372486 ]
 [-1.2519338   0.02122428 -0.5513758  -1.7431698 ]]...
S1:[[ 1.6243454  0.         0.         0.       ]
 [ 0.        -0.6117564  0.         0.       ]
 [ 0.         0.        -0.5281718  0.       ]
 [ 0.         0.         0.        -1.0729686]]...
Z0:[[-1.1012203   1.5457517   0.383644  ]
 [-0.87965786 -1.2246722  -0.9811211 ]
 [ 0.08780783 -0.20326038 -0.5581562 ]
 [-0.72054404 -0.6259924  -0.71502596]
 [-0.34835446 -0.33646983  0.18257578]]...
Z1:[[ 1.7601889e-01  4.4997773e-01 -1.3233465e+00 -3.7439287e+00]
 [ 2.0465101e-03  1.5354780e+00  1.5323893e+00  1.8048377e+00]
 [ 5.8949625e-01 -7.9306550e-03  4.5754948e-01  1.3011527e+00]
 [ 1.5916619e-01  1.0750805e+00  9.1810775e-01  9.3267345e-01]
 [-6.0847604e-01  5.4752207e-01  1.7844909e-01 -4.3485320e-01]]...
Y_pred:[[ 2.8591549e-01 -2.7527675e-01  6.9895428e-01  4.0171180e+00]
 [ 3.3242393e-03 -9.3933845e-01 -8.0936480e-01 -1.9

### Two-layer network with diagonal matrix activation functions
That worked. So let's wrap-up with a two-layer network.

In [43]:
np.random.seed(1) # for consistency
tf.random.set_seed(1) # for consistency
n = 15
fL = 4
Z0 = tf.random.normal(shape=(n,2))
W1 = tf.Variable(tf.random.normal(shape=(2,3)), dtype=tf.float32)
S1 = tf.cast(tf.linalg.diag(np.random.normal(size=3)), dtype=tf.float32)
W2 = tf.Variable(tf.random.normal(shape=(3,fL)), dtype=tf.float32)
S2 = tf.cast(tf.linalg.diag(np.random.normal(size=fL)), dtype=tf.float32)
Y = tf.random.normal(shape=(n,fL))
loss_fn = tf.keras.losses.MeanSquaredError()  # equivalent to tf.reduce_mean((Y_pred - Y)**2, axis=None) = 1/(N * f_L) * sum((Y_pred-Y)**2)

with tf.GradientTape(persistent=True) as tape:
  Z1 = tf.matmul(Z0, W1)
  A1 = tf.matmul(Z1, S1)
  Z2 = tf.matmul(A1, W2)
  A2 = tf.matmul(Z2, S2)
  Y_pred = A2
  loss = loss_fn(Y, Y_pred)

print(f"Z0:{Z0[:5,:]}...")
print(f"Z1:{Z1[:5,:]}...")
print(f"Y_pred:{Y_pred[:5,:]}...")
print(f"error:{(Y_pred - Y)[:5,:]}...")
print(f"loss:{loss}")
print("---expect---")
dZ1dW1 = Z0.numpy().T
dZ2dW2 = A1.numpy().T
dJdA2 = 2/(n*fL)*(Y_pred-Y)
dA2dZ2 = S2.numpy().T
dZ2dA1 = W2.numpy().T
dA1dZ1 = S1.numpy().T
print(f"dZ1/dW1 = Z0.T: {dZ1dW1[:,:5]}...<more cols>")
print(f"dJ/dA2 = 2(n*fL)*error: {dJdA2[:5,:]}...")
print(f"dA2/dZ2 = S2.T: {dA2dZ2}")
print(f"dZ2/dA1 = W2.T: {dZ2dA1}")
print(f"dA1/dZ1 = S1.T: {dA1dZ1}")
print(f"dJ/dW1 = dZ1dW1.dJ/dA2.dA2/dZ2.dZ2dA1.dA1dZ1: {np.matmul(dZ1dW1, np.matmul(dJdA2, np.matmul(np.matmul(dA2dZ2, dZ2dA1), dA1dZ1)))}")
print(f"dJ/dW2 = dZ2dW2.dJ/dA2.dA2/dZ2: {np.matmul(dZ2dW2, np.matmul(dJdA2, dA2dZ2))}")
print("---actual---")
print(f"dJ/dA2: {tape.gradient(loss, Y_pred)[:5,:]}...")
print(f"dJ/dW1: {tape.gradient(loss, W1)}")
print(f"dJ/dW2: {tape.gradient(loss, W2)}")

Z0:[[-1.1012203   1.5457517 ]
 [ 0.383644   -0.87965786]
 [-1.2246722  -0.9811211 ]
 [ 0.08780783 -0.20326038]
 [-0.5581562  -0.72054404]]...
Z1:[[ 1.6220962   2.2983549  -0.6868335 ]
 [-1.0210704  -1.0435181   0.4061985 ]
 [-1.8049746   0.634146    0.5573204 ]
 [-0.23627473 -0.24020937  0.09391228]
 [-1.1880339   0.09443104  0.38776952]]...
Y_pred:[[  1.3818731   -2.3498108  -11.148681    -2.6942422 ]
 [ -0.8016453    1.2588058    5.968388     1.9256067 ]
 [ -0.99778616   0.8707299    4.099388     4.815632  ]
 [ -0.18528685   0.29059806   1.3778011    0.446301  ]
 [ -0.711374     0.7495918    3.5387316    2.9857194 ]]...
error:[[-0.31214356 -2.469504   -9.990221   -2.8668463 ]
 [-0.08714896  0.56920516  7.059204    3.112117  ]
 [ 0.30440176 -0.3532722   4.076428    6.005748  ]
 [ 2.2487893   1.3300612   0.4525537   0.19529685]
 [-1.9010997   0.26948476  3.4185803   3.7106118 ]]...
loss:11.836978912353516
---expect---
dZ1/dW1 = Z0.T: [[-1.1012203   0.383644   -1.2246722   0.08780783 -0

Amazing!! I actually got it to work. Phew!

😊👌🙌

## Gradients across layers
Now that we've got the basics down pat we can start to explore more, and we can do this more efficiently by using TF to do the calculations for us - knowing the limitations of what it's good for and what it's not. Specifically:
* TF is good at gradients of the loss w.r.t. variables
* TF is not good at gradients of in-between partials

First, we'll get a baseline by looking at how the weight gradients change across the layers and with increasing numbers of layers without activation functions. We'll calibrate our networks s.t. the error is $\frac{1}{nf_L}(\hat{Y} - Y)$ is $\vec{1}$ for every row.

### Representative
This shows the full steps for a 2-layer network.

In [55]:
n = 15
fL = 4
Z0 = tf.constant(np.ones(shape=(n,2)), dtype=tf.float32)
W1 = tf.Variable(np.ones(shape=(2,3)), dtype=tf.float32)
W2 = tf.Variable(np.ones(shape=(3,fL)), dtype=tf.float32)
expected_Y_values = Z0.shape[1] * W1.shape[1]
Y = tf.constant(np.ones(shape=(n,fL)) * (expected_Y_values - 1), dtype=tf.float32)
loss_fn = tf.keras.losses.MeanSquaredError()

with tf.GradientTape(persistent=True) as tape:
  Z1 = tf.matmul(Z0, W1)
  Z2 = tf.matmul(Z1, W2)
  Y_pred = Z2
  loss = loss_fn(Y, Y_pred)

print(f"Z0:{Z0[:5,:]}...")
print(f"Z1:{Z1[:5,:]}...")
print(f"Y_pred:{Y_pred[:5,:]}...")
print(f"error:{(Y_pred - Y)[:5,:]}...")
print(f"loss:{loss}")
print("---actual---")
print(f"2/(nfL)*1: {2/(n*fL)}")
print(f"dJ/dA2: {tape.gradient(loss, Y_pred)[:5,:]}...")
print(f"dJ/dW1: {tape.gradient(loss, W1)}")
print(f"dJ/dW2: {tape.gradient(loss, W2)}")

Z0:[[1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]]...
Z1:[[2. 2. 2.]
 [2. 2. 2.]
 [2. 2. 2.]
 [2. 2. 2.]
 [2. 2. 2.]]...
Y_pred:[[6. 6. 6. 6.]
 [6. 6. 6. 6.]
 [6. 6. 6. 6.]
 [6. 6. 6. 6.]
 [6. 6. 6. 6.]]...
error:[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]...
loss:1.0
---actual---
2/(nfL)*1: 0.03333333333333333
dJ/dA2: [[0.03333334 0.03333334 0.03333334 0.03333334]
 [0.03333334 0.03333334 0.03333334 0.03333334]
 [0.03333334 0.03333334 0.03333334 0.03333334]
 [0.03333334 0.03333334 0.03333334 0.03333334]
 [0.03333334 0.03333334 0.03333334 0.03333334]]...
dJ/dW1: [[2. 2. 2.]
 [2. 2. 2.]]
dJ/dW2: [[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]


### Various layered networks without activation function
The following shows the gradients for each weight layer, where the error has been calibrated to $\vec{1}$, with the loss partial as $2/(nf_L)*1$.

In [92]:
n = 15
f = [3, 3, 3, 3, 3, 3]

for L in range(1,len(f)):
  print(f"---{L}-layer network---")
  fL = f[L]
  W = [0] * (L+1)
  Z = [0] * (L+1)
  Z[0] = tf.constant(np.ones(shape=(n,f[0])), dtype=tf.float32)

  for l in range(1,L+1):
    W[l] = tf.Variable(np.ones(shape=(f[l-1],f[l])), dtype=tf.float32)

  expected_Y_values = Z[0].shape[1]
  for l in range(1,L):
    expected_Y_values *= W[l].shape[1]
  expected_error = 1

  Y = tf.constant(np.ones(shape=(n,fL)) * (expected_Y_values - expected_error), dtype=tf.float32)
  loss_fn = tf.keras.losses.MeanSquaredError()

  with tf.GradientTape(persistent=True) as tape:
    for l in range(1,L+1):
      Z[l] = tf.matmul(Z[l-1], W[l])
    Y_pred = Z[L]
    loss = loss_fn(Y, Y_pred)

  #print(f"(2/nfL)*1: {2/(n*fL)}")
  #print(f"dJ/dZL: {tape.gradient(loss, Z[L])[:5,:]}...")
  for l in range(1,L+1):
    print(f"dJ/dW{l}: {tape.gradient(loss, W[l])}")

---1-layer network---
dJ/dW1: [[0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]]
---2-layer network---
dJ/dW1: [[2. 2. 2.]
 [2. 2. 2.]
 [2. 2. 2.]]
dJ/dW2: [[2. 2. 2.]
 [2. 2. 2.]
 [2. 2. 2.]]
---3-layer network---
dJ/dW1: [[6.000001 6.000001 6.000001]
 [6.000001 6.000001 6.000001]
 [6.000001 6.000001 6.000001]]
dJ/dW2: [[6.0000005 6.0000005 6.0000005]
 [6.0000005 6.0000005 6.0000005]
 [6.0000005 6.0000005 6.0000005]]
dJ/dW3: [[6.0000005 6.0000005 6.0000005]
 [6.0000005 6.0000005 6.0000005]
 [6.0000005 6.0000005 6.0000005]]
---4-layer network---
dJ/dW1: [[18. 18. 18.]
 [18. 18. 18.]
 [18. 18. 18.]]
dJ/dW2: [[18.000002 18.000002 18.000002]
 [18.000002 18.000002 18.000002]
 [18.000002 18.000002 18.000002]]
dJ/dW3: [[18.000002 18.000002 18.000002]
 [18.000002 18.000002 18.000002]
 [18.000002 18.000002 18.000002]]
dJ/dW4: [[18. 18. 18.]
 [18. 18. 18.]
 [18. 18. 18.]]
---5-layer network---
dJ/dW1: [[54. 54. 54.]
 [54. 54. 54.]
 [54. 54. 54.]]

Notice that the component values of the gradients increase rapidly with increased depth. This shows the importance of correctly initialising weights.

In this case, with 3x3 weight matrices, their gradient component values follow the formula:
$$\frac{2}{f_L}\prod_{l=0}^{L-1}f_l$$.

Let's try the same thing but with better weight matrice normalisation.

In [112]:
n = 15
f = [3, 3, 3, 3, 3, 3]

for L in range(1,len(f)):
  print(f"---{L}-layer network---")
  fL = f[L]
  W = [0] * (L+1)
  Z = [0] * (L+1)
  Z[0] = tf.constant(np.ones(shape=(n,f[0])), dtype=tf.float32)

  for l in range(1,L+1):
    W[l] = tf.Variable(np.ones(shape=(f[l-1],f[l])) / f[l], dtype=tf.float32)

  Y = tf.constant(np.ones(shape=(n,fL)) - 1, dtype=tf.float32)
  loss_fn = tf.keras.losses.MeanSquaredError()

  with tf.GradientTape(persistent=True) as tape:
    for l in range(1,L+1):
      Z[l] = tf.matmul(Z[l-1], W[l])
    Y_pred = Z[L]
    loss = loss_fn(Y, Y_pred)

  #print(f"Y_pred: {Y_pred[:5,:]}...")
  #print(f"loss: {loss}")
  #print(f"Y_pred - Y: {(Y_pred - Y)[:5,:]}...")
  #print(f"(2/nfL)*1: {2/(n*fL)}")
  #print(f"dJ/dZL: {tape.gradient(loss, Z[L])[:5,:]}...")
  for l in range(1,L+1):
    print(f"dJ/dW{l}: {tape.gradient(loss, W[l])}")

---1-layer network---
dJ/dW1: [[0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]]
---2-layer network---
dJ/dW1: [[0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]]
dJ/dW2: [[0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]]
---3-layer network---
dJ/dW1: [[0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]]
dJ/dW2: [[0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]]
dJ/dW3: [[0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]]
---4-layer network---
dJ/dW1: [[0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]]
dJ/dW2: [[0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]]
dJ/dW3: [[0.6666667 0.6666667 0.6666667]
 [0.666

Great. Now we have a strategy for generating consistent-valued gradients for different network depths.

Next, we add activation....

### Various-layered networks with activation function
Just to make sure we can do this, we add a simple $\mathbb{I}$ activation function at all layers. This is redundant for now but proves out the code.

In [118]:
n = 15
f = [3, 3, 3, 3, 3, 3]

for L in range(1,len(f)):
#for L in range(2,3):
  print(f"---{L}-layer network---")
  fL = f[L]
  W = [0] * (L+1)
  S = [0] * (L+1)
  Z = [0] * (L+1)
  A = [0] * (L+1)
  A[0] = tf.constant(np.ones(shape=(n,f[0])), dtype=tf.float32)

  for l in range(1,L+1):
    W[l] = tf.Variable(np.ones(shape=(f[l-1],f[l])) / f[l], dtype=tf.float32)
    S[l] = tf.constant(np.eye(f[l]), dtype=tf.float32)

  Y = tf.constant(np.ones(shape=(n,fL)) - 1, dtype=tf.float32)
  loss_fn = tf.keras.losses.MeanSquaredError()

  with tf.GradientTape(persistent=True) as tape:
    for l in range(1,L+1):
      Z[l] = tf.matmul(A[l-1], W[l])
      A[l] = tf.matmul(Z[l], S[l])
    Y_pred = A[L]
    loss = loss_fn(Y, Y_pred)

  #print(f"Y_pred: {Y_pred[:5,:]}...")
  #print(f"loss: {loss}")
  #print(f"Y_pred - Y: {(Y_pred - Y)[:5,:]}...")
  #print(f"(2/nfL)*1: {2/(n*fL)}")
  #print(f"dJ/dZL: {tape.gradient(loss, Z[L])[:5,:]}...")
  for l in range(1,L+1):
    print(f"dJ/dW{l}: {tape.gradient(loss, W[l])}")
    #print(f"dJ/dA{l}: {tape.gradient(loss, A[l])}")

---1-layer network---
dJ/dW1: [[0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]]
---2-layer network---
dJ/dW1: [[0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]]
dJ/dW2: [[0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]]
---3-layer network---
dJ/dW1: [[0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]]
dJ/dW2: [[0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]]
dJ/dW3: [[0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]]
---4-layer network---
dJ/dW1: [[0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]]
dJ/dW2: [[0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]
 [0.6666667 0.6666667 0.6666667]]
dJ/dW3: [[0.6666667 0.6666667 0.6666667]
 [0.666

Now the real thing.
For this, we'll leave the activations as $\mathbb{I}$ for most layers. We'll attempt to pick a middle layer, and for that one layer we'll zero-out the middle activation value only.

In [125]:
n = 15
f = [3, 3, 3, 3, 3, 3, 3, 3]

for L in range(1,len(f)):
#for L in range(2,3):
  print(f"---{L}-layer network---")
  fL = f[L]
  W = [0] * (L+1)
  S = [0] * (L+1)
  Z = [0] * (L+1)
  A = [0] * (L+1)
  A[0] = tf.constant(np.ones(shape=(n,f[0])), dtype=tf.float32)

  for l in range(1,L+1):
    W[l] = tf.Variable(np.ones(shape=(f[l-1],f[l])) / f[l], dtype=tf.float32)
    S[l] = tf.constant(np.eye(f[l]), dtype=tf.float32)
    # zero out middle activation of middle layer
    if l == math.ceil(L/2.):
      diag = [1.] * f[l]
      diag[f[l] // 2] = 0.
      S[l] = tf.constant(tf.linalg.diag(diag))
      print(f"S{l}: {S[l]}")

  Y = tf.constant(np.ones(shape=(n,fL)) - 1, dtype=tf.float32)
  loss_fn = tf.keras.losses.MeanSquaredError()

  with tf.GradientTape(persistent=True) as tape:
    for l in range(1,L+1):
      Z[l] = tf.matmul(A[l-1], W[l])
      A[l] = tf.matmul(Z[l], S[l])
    Y_pred = A[L]
    loss = loss_fn(Y, Y_pred)

  #print(f"Y_pred: {Y_pred[:5,:]}...")
  #print(f"loss: {loss}")
  #print(f"Y_pred - Y: {(Y_pred - Y)[:5,:]}...")
  #print(f"(2/nfL)*1: {2/(n*fL)}")
  #print(f"dJ/dZL: {tape.gradient(loss, Z[L])[:5,:]}...")
  for l in range(1,L+1):
    print(f"dJ/dW{l}: {tape.gradient(loss, W[l])}")
    #print(f"dJ/dA{l}: {tape.gradient(loss, A[l])}")

---1-layer network---
S1: [[1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 1.]]
dJ/dW1: [[0.6666667 0.        0.6666667]
 [0.6666667 0.        0.6666667]
 [0.6666667 0.        0.6666667]]
---2-layer network---
S1: [[1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 1.]]
dJ/dW1: [[0.44444445 0.         0.44444445]
 [0.44444445 0.         0.44444445]
 [0.44444445 0.         0.44444445]]
dJ/dW2: [[0.44444445 0.44444445 0.44444445]
 [0.         0.         0.        ]
 [0.44444445 0.44444445 0.44444445]]
---3-layer network---
S2: [[1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 1.]]
dJ/dW1: [[0.2962963 0.2962963 0.2962963]
 [0.2962963 0.2962963 0.2962963]
 [0.2962963 0.2962963 0.2962963]]
dJ/dW2: [[0.44444445 0.         0.44444445]
 [0.44444445 0.         0.44444445]
 [0.44444445 0.         0.44444445]]
dJ/dW3: [[0.44444445 0.44444445 0.44444445]
 [0.         0.         0.        ]
 [0.44444445 0.44444445 0.44444445]]
---4-layer network---
S2: [[1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 1.]]
dJ/dW1: [[0.2962963 0.2962963 0.2962963]
 [0.2962963 0.2962963 0